# Pre requisites

In [1]:
!nvidia-smi

Mon Jul 19 06:16:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d tapakah68/supervisely-filtered-segmentation-person-dataset

!unzip -q /content/supervisely-filtered-segmentation-person-dataset.zip

kaggle.json
100% 4.30G/4.31G [01:25<00:00, 36.9MB/s]
100% 4.31G/4.31G [01:25<00:00, 54.1MB/s]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!pip install q tensorflow==2.1
!pip install q keras==2.3.1
!pip install git+https://github.com/qubvel/segmentation_models

# for str decode error ... run it and restart runtime
!pip uninstall h5py
!pip install h5py==2.10.0

# Preprocessing and Model code

In [20]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf


def load_dataset(dataset_path):
    images = glob(os.path.join(dataset_path, "images/*"))
    masks = glob(os.path.join(dataset_path, "masks/*"))

    train_x, test_x = train_test_split(images, test_size=0.2, random_state=1996)
    train_y, test_y = train_test_split(masks, test_size=0.2, random_state=1996)

    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = load_dataset('/content/supervisely_person_clean_2667_img')
print(len(train_x), len(train_y), len(test_x), len(test_y))

train_X_y_paths = list(zip(train_x, train_y))
val_X_y_paths = list(zip(test_x, test_y))

def shuffle(samples):
  index = np.random.permutation(len(samples))
  return np.array(samples)[index]

# def get_data(base_dir,w):
#   img = glob(base_dir+f'images/{w}/*/*.png')
#   img_names = [path.split(f'/{w}/')[1].split('_leftImg8bit.png')[0] for path in img]
#   label = [base_dir + f'gtFine/{w}/' + name + '_gtFine_color.png' for name in img_names]
#   del img_names
#   return img, label

# base_dir = "/content/people_segmentation"
# train_img,train_label = pre_shuffle(*get_data(base_dir,"images"))
# val_img,val_label = pre_shuffle(*get_data(base_dir,"val"))

def get_data_generator(samples,batch_size):

  while True:
    for offset in range(0, len(samples), batch_size): 
      samples = shuffle(samples)           
      batch_samples = samples[offset:offset+batch_size]
      X_train = []
      Y_train = []        
 
      for batch_sample in batch_samples:
          
        X_image = cv2.imread(batch_sample[0])[:,:,::-1]# convert bgr to rgb
        X_image = cv2.resize(X_image, (512, 512))
        X_image = X_image / 255.0
        X_image = X_image.astype(np.float32)

        Y_image = cv2.imread(batch_sample[1], 0)
#         Y_image = Y_image * 255.0
        Y_image = cv2.resize(Y_image, (512, 512))
        Y_image = Y_image / 255.0
        Y_image = Y_image.astype(np.float32)
        Y_image = np.expand_dims(Y_image, axis=-1)

        X_train.append(X_image)
        Y_train.append(Y_image)
      yield np.array(X_train), np.array(Y_train)


BATCH_SIZE = 12
train_generator = get_data_generator(train_X_y_paths,batch_size=BATCH_SIZE)
val_generator = get_data_generator(val_X_y_paths,batch_size=BATCH_SIZE)
 
x_train, y_train = next(train_generator)
x_val, y_val  = next(val_generator)
 
print(x_train.shape,y_train.shape)
print(x_val.shape, y_val.shape)

2133 2133 534 534
(12, 512, 512, 3) (12, 512, 512, 1)
(12, 512, 512, 3) (12, 512, 512, 1)


In [29]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_mobilenetv2_unet(input_shape, classes = 1):    ## (512, 512, 3)
    """ Input """
    inputs = Input(shape=input_shape)
    """ Pre-trained MobileNetV2 """
    encoder = MobileNetV2(include_top=False, weights="imagenet", input_tensor=inputs, alpha=1.0)
    """ Encoder """
    s1 = encoder.get_layer("input_8").output                ## (512 x 512)
    s2 = encoder.get_layer("block_1_expand_relu").output    ## (256 x 256)
    s3 = encoder.get_layer("block_3_expand_relu").output    ## (128 x 128)
    s4 = encoder.get_layer("block_6_expand_relu").output    ## (64 x 64)
    """ Bridge """
    b1 = encoder.get_layer("block_13_expand_relu").output   ## (32 x 32)
    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                         ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                         ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                         ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                          ## (512 x 512)
    """ Output """
    outputs = Conv2D(classes, (1, 1), padding="same", activation="sigmoid")(d4)
    model = Model(inputs, outputs, name="MobileNetV2_U-Net")
    return model

model = build_mobilenetv2_unet((512, 512, 3), classes = 1)
# model.summary()

TF Version:  2.1.0


/usr/local/lib/python3.7/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "MobileNetV2_U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 513, 513, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 256, 256, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 256, 256, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________

# Strating training

In [24]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
%env SM_FRAMEWORK=tf.keras

import numpy as np
import tensorflow as tf
# from model import build_unet
# from data import load_dataset, tf_dataset
from segmentation_models.losses import bce_jaccard_loss, dice_loss, JaccardLoss
from segmentation_models.metrics import iou_score, f1_score, precision, recall
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

""" Hyperparamaters """
dataset_path = "supervisely_person_clean_2667_img"
input_shape = (512, 512, 3)
batch_size = 12
epochs = 100
lr = 1e-2
# model_path = "unet.h5"
# csv_path = "data.csv"

""" Load the dataset """
# (train_x, train_y), (test_x, test_y) = load_dataset(dataset_path)
# print(f"Train: {len(train_x)} - {len(train_y)}")
# print(f"Test: {len(test_x)} - {len(test_y)}")

# train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
# test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

""" callbacks """
checkpoint_filepath = '/content/Unet_MobilenetV2_resnet50_Human_Seg_000+{epoch}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_iou_score',
    mode='max',
    verbose = 1,
    period = 5,
    save_best_only=False
    )

callbacks = [
    model_checkpoint_callback,
    ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1),
#     CSVLogger(csv_path),
    EarlyStopping(monitor="val_loss", patience=10)
]

""" steps per epochs """
# train_steps = len(train_x)//batch_size
# if len(train_x) % batch_size != 0:
#     train_steps += 1

# test_steps = len(test_x)//batch_size
# if len(test_x) % batch_size != 0:
#     test_steps += 1

# print(train_steps, test_steps)


""" Model training """
ls = dice_loss + bce_jaccard_loss
metrics = [precision, recall, f1_score, iou_score] 

for layer in model.layers:
  # if layer.name == "decoder_conv0_depthwise":
  #   break
  # else:
  layer.trainable = True
# for layer in model.layers:
#   print(layer.name,layer.trainable)

model.compile(
    loss=ls,
    optimizer=tf.keras.optimizers.Adam(lr),
    metrics=metrics
)
# model.summary()

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks
)


env: SM_FRAMEWORK=tf.keras
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 178 steps, validate for 45 steps
Epoch 1/100
178/178 [==============================] - 418s 2s/step - loss: 1.1215 - precision: 0.6940 - recall: 0.7581 - f1-score: 0.7168 - iou_score: 0.5660 - val_loss: 3.8328 - val_precision: 0.3239 - val_recall: 0.0337 - val_f1-score: 0.0609 - val_iou_score: 0.0314
Epoch 2/100
178/178 [==============================] - 418s 2s/step - loss: 0.8551 - precision: 0.7795 - recall: 0.8241 - f1-score: 0.7950 - iou_score: 0.6617 - val_loss: 3.4974 - val_precision: 0.3578 - val_recall: 0.1597 - val_f1-score: 0.2191 - val_iou_score: 0.1232
Epoch 3/100
178/178 [==============================] - 409s 2s/step - loss: 0.7320 - precision: 0.8189 - recall: 0.8518 - f1-score: 0.8284 - iou_score: 0.7093 - val_loss: 3.0276 - val_precision: 0.2993 - val_recall: 0.2147 - val_f1-score: 0.2474 - val_iou_score: 0.1414
Epoch 4/100
178/178 [==============================] - 402s 2s/step - 

KeyboardInterrupt: ignored

# Predictions

In [ ]:
#@title
model.load_weights('/content/Unet_MobilenetV2_resnet50_Human_Seg_000+25.h5')
model.compile(
    loss=ls,
    # optimizer=tf.keras.optimizers.Adam(lr),
    metrics=metrics
)

In [47]:
#@title
img = cv2.imread('SJ.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [48]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [49]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_SJ.jpg',pred)

True

In [50]:
#@title
img = cv2.imread('SJ.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_SJ.jpg',overlay)

True

In [ ]:
#@title


In [43]:
#@title
img = cv2.imread('75805310.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [44]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [45]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_75805310.jpg',pred)

True

In [46]:
#@title
img = cv2.imread('75805310.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_75805310.jpg',overlay)

True

In [ ]:
#@title


In [51]:
#@title
img = cv2.imread('oldestman.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [52]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [53]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_oldestman.jpg',pred)

True

In [54]:
#@title
img = cv2.imread('oldestman.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_oldestman.jpg',overlay)

True

In [ ]:
#@title


In [55]:
#@title
img = cv2.imread('multiple.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [56]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [57]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_multiple.jpg',pred)

True

In [58]:
#@title
img = cv2.imread('multiple.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_multiple.jpg',overlay)

True

In [ ]:
#@title


In [59]:
#@title
img = cv2.imread('2.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [60]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [61]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_2.jpg',pred)

True

In [62]:
#@title
img = cv2.imread('2.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_2.jpg',overlay)

True

# Retraining from 25 epochs to 100

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
%env SM_FRAMEWORK=tf.keras

import numpy as np
import tensorflow as tf
# from model import build_unet
# from data import load_dataset, tf_dataset
from segmentation_models.losses import bce_jaccard_loss, dice_loss, JaccardLoss
from segmentation_models.metrics import iou_score, f1_score, precision, recall
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping

""" Hyperparamaters """
dataset_path = "supervisely_person_clean_2667_img"
input_shape = (512, 512, 3)
batch_size = 12
epochs = 75
lr = 1e-2
# model_path = "unet.h5"
# csv_path = "data.csv"

""" Load the dataset """
# (train_x, train_y), (test_x, test_y) = load_dataset(dataset_path)
# print(f"Train: {len(train_x)} - {len(train_y)}")
# print(f"Test: {len(test_x)} - {len(test_y)}")

# train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
# test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

""" callbacks """
checkpoint_filepath = '/content/Unet_MobilenetV2_Human_Seg_025+{epoch}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_iou_score',
    mode='max',
    verbose = 1,
    period = 5,
    save_best_only=False
    )

callbacks = [
    model_checkpoint_callback,
    ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1),
#     CSVLogger(csv_path),
    EarlyStopping(monitor="val_loss", patience=10)
]

""" steps per epochs """
# train_steps = len(train_x)//batch_size
# if len(train_x) % batch_size != 0:
#     train_steps += 1

# test_steps = len(test_x)//batch_size
# if len(test_x) % batch_size != 0:
#     test_steps += 1

# print(train_steps, test_steps)


""" Model training """
ls = dice_loss + bce_jaccard_loss
metrics = [precision, recall, f1_score, iou_score] 

for layer in model.layers:
  # if layer.name == "decoder_conv0_depthwise":
  #   break
  # else:
  layer.trainable = True
# for layer in model.layers:
#   print(layer.name,layer.trainable)

from tensorflow.keras.models import load_model
model = load_model('/content/Unet_MobilenetV2_resnet50_Human_Seg_000+25.h5', compile = False)
model.compile(
    loss=ls,
    # optimizer=tf.keras.optimizers.Adam(lr),
    metrics=metrics
)
# model.summary()

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks
)


env: SM_FRAMEWORK=tf.keras
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 178 steps, validate for 45 steps
Epoch 1/75
178/178 [==============================] - 416s 2s/step - loss: 0.1830 - precision: 0.9585 - recall: 0.9649 - f1-score: 0.9615 - iou_score: 0.9261 - val_loss: 0.5633 - val_precision: 0.9475 - val_recall: 0.8286 - val_f1-score: 0.8831 - val_iou_score: 0.7922
Epoch 2/75
178/178 [==============================] - 410s 2s/step - loss: 0.1805 - precision: 0.9598 - recall: 0.9654 - f1-score: 0.9624 - iou_score: 0.9277 - val_loss: 0.5512 - val_precision: 0.9509 - val_recall: 0.8315 - val_f1-score: 0.8864 - val_iou_score: 0.7974
Epoch 3/75
178/178 [==============================] - 414s 2s/step - loss: 0.1702 - precision: 0.9625 - recall: 0.9670 - f1-score: 0.9645 - iou_score: 0.9317 - val_loss: 0.5557 - val_precision: 0.9405 - val_recall: 0.8519 - val_f1-score: 0.8930 - val_iou_score: 0.8081
Epoch 4/75
178/178 [==============================] - 415s 2s/step - loss